# Esercizio 1 - Definizioni

Lo scopo dell'esercizio è determinare la similarità delle definizioni date dagli studenti per i seguenti termini:

|  | Generico | Specifico |
|:--------:|:--------:|:---------:|
| **Concreto** | Person | Brick |
| **Astratto** | Emotion | Revenge |


In [5]:
import pandas as pd
import nltk
from collections import Counter

### Leggi il file di input

Salvo le varie definizioni degli studenti in un dizionario con:
- chiave: parola (in ordine Emotion, Person, Revenge, Brick)
- valore: lista di definizioni date dagli studenti

In [6]:
df = pd.read_excel('../data/definizioni.xlsx')
students = df.columns[1:].tolist()
words = df[df.columns[0]].tolist()
def_dict = {}
for definition in words:
    def_dict[definition] = []
for student in students:
    for definition in words:
        def_dict[definition].append(df[student][df[df.columns[0]] == definition].values[0])

# def_dict

### Ripulisci il dictionary delle definizioni

Salvo le definizioni pulite in un dizionario strutturato come il precedente

In [7]:
stopwords = nltk.corpus.stopwords.words('english')
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
lemmatizer = nltk.stem.WordNetLemmatizer()

# Tokenize and lemmatize definitions and remove stopwords and punctuation
def clean_definitions(dict):
    cleaned = dict.copy()
    for key in cleaned:
        definitions = [d for d in cleaned[key] if d == d]
        tmp = []
        for definition in definitions:
            tmp.append([lemmatizer.lemmatize(token.lower()) for token in tokenizer.tokenize(definition) if token.lower() not in stopwords])
        cleaned[key] = tmp
    return cleaned

clean_def_dict = clean_definitions(def_dict)

### Calcola la similarità tra le definizioni

Per ogni parola, calcolo le *n* parole più frequenti nelle definizioni date dagli studenti, usando la funzione `most_common` di `Counter`.  
Per calcolare la similarità tra le definizioni, calcolo la media delle volte che le parole più frequenti compaiono nelle definizioni, data da:  
numero di volte che le parole più frequenti compaiono in una definizione / numero di definizioni.  
Infine la similarità totale è data da:  
somma delle medie di ogni parola frequente / numero di parole frequenti.

In [8]:
# Get the most common words in the definitions of a word
def get_common_words(dict, word, n):
    words = []
    for definition in dict[word]:
        words += definition
    return Counter(words).most_common(n)

# Get mean number of times a frequent word appears in the definitions of a word
def get_mean_freq(dict, word, freq_word):
    return freq_word[1]/len(dict[word])

# Get total similarity of definitions of a word as the overall mean of frequent words
def similarity(dict, word, n):
    freq_words = get_common_words(dict, word, n)
    mean = 0
    for freq_word in freq_words:
        mean += get_mean_freq(dict, word, freq_word)
    return mean/n

# Print the similarity of each word
n_freq_words = 5
print(f'Considering the {n_freq_words} most frequent words')
for word in clean_def_dict:
    print(f'Word: {word}')
    print(f'Similarity: {similarity(clean_def_dict, word, n_freq_words)}\n')

Considering the 5 most frequent words
Word: Emotion
Similarity: 0.2733333333333333

Word: Person
Similarity: 0.2903225806451612

Word: Revenge
Similarity: 0.2733333333333333

Word: Brick
Similarity: 0.5483870967741935

